# Sommaire :
- <a href="#C1">Import des librairies</a>
- <a href="#C2">Import des données</a>
- <a href="#C3">Lecture des données & suppression granularités inutiles</a>
- <a href="#C4">Correspondance WaterServices et Mortality</a>
- <a href="#C5">Correspondance df et PoliticalStability</a>
- <a href="#C6">Correspondance df & Population</a>
- <a href="#C7">Correspondance df & RegionCountry</a>
- <a href="#C8">Fusion des datasets après nettoyage</a>
    - <a href="#C9"> Fusion WaterServices & Mortality </a>
    - <a href="#C10"> Fusion df & Political Stability </a>
    - <a href="#C11"> Fusion df1 & Population </a>
    - <a href="#C12"> Fusion df2 & Population </a>
- <a href="#C14.2">Ajout de la colonne Urban population rate</a>
- <a href="#C14.5"> Ajout des densités de Population </a>
    - <a href="#C14.51"> Ajout des densités de Population par Pays</a>
    - <a href="#C14.52"> Ajout des densités de Population par Région</a>
    - <a href="#C14.53"> Ajout de la densité de Population niveau Monde</a>
- <a href="#C15"> Export du .csv </a>


# <a name="C1"> Import des librairies </a>

In [318]:
import pandas as pd
pd.options.display.max_columns = None # pour afficher toutes les colonnes
pd.options.display.max_rows = None # pour afficher toutes les lignes

import numpy as np

from scipy import stats

from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

import seaborn as sns

import statsmodels.api as sm
import pylab

# <a name="C2"> Import des données </a>

Données d'OpenClassrooms et de la Banque mondiale pour les surfaces des pays

In [319]:
Population = pd.read_csv('Population.csv')
BasicAndSafelyManagedDrinkingWaterServices = pd.read_csv('BasicAndSafelyManagedDrinkingWaterServices.csv')
PoliticalStability = pd.read_csv('PoliticalStability.csv')
MortalityRateAttributedToWater = pd.read_csv('MortalityRateAttributedToWater.csv')
RegionCountry = pd.read_csv('RegionCountry.csv')

# <a name="C3"> Lecture des données & suppression granularités inutiles </a>

## BasicAndSafelyManagedDrinkingWaterServices

In [320]:
BasicAndSafelyManagedDrinkingWaterServices.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)
0,2000,Afghanistan,Rural,21.61913,NaN
1,2000,Afghanistan,Total,27.77190,NaN
2,2000,Afghanistan,Urban,49.48745,NaN
3,2000,Albania,Rural,81.78472,NaN
4,2000,Albania,Total,87.86662,49.29324


In [321]:
len(BasicAndSafelyManagedDrinkingWaterServices)

10476

In [322]:
BasicAndSafelyManagedDrinkingWaterServices['Granularity'].value_counts()

Urban    3492
Rural    3492
Total    3492
Name: Granularity, dtype: int64

## MortalityRateAttributedToWater

In [323]:
MortalityRateAttributedToWater.head()

,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
0,2016,Afghanistan,Female,15.31193,NaN
1,2016,Afghanistan,Male,12.61297,NaN
2,2016,Afghanistan,Total,13.92067,4824.353
3,2016,Albania,Female,0.12552,NaN
4,2016,Albania,Male,0.20650,NaN


In [324]:
len(MortalityRateAttributedToWater)

549

In [325]:
MortalityRateAttributedToWater['Granularity'].value_counts()

Total     183
Male      183
Female    183
Name: Granularity, dtype: int64

Suppression des granularités Male/Female :

In [326]:
MortalityRateAttributedToWater.drop\
(MortalityRateAttributedToWater[MortalityRateAttributedToWater['Granularity'] == 'Male'].index,\
 inplace=True)
MortalityRateAttributedToWater.drop\
(MortalityRateAttributedToWater[MortalityRateAttributedToWater['Granularity'] == 'Female'].index,\
 inplace=True)

MortalityRateAttributedToWater['Granularity'].value_counts() # vérification

Total    183
Name: Granularity, dtype: int64

## PoliticalStability

In [327]:
PoliticalStability.head()

,Country,Year,Political_Stability,Granularity
0,Afghanistan,2000,-2.44,Total
1,Afghanistan,2002,-2.04,Total
2,Afghanistan,2003,-2.20,Total
3,Afghanistan,2004,-2.30,Total
4,Afghanistan,2005,-2.07,Total


In [328]:
len(PoliticalStability)

3526

In [329]:
PoliticalStability['Granularity'].value_counts()

Total    3526
Name: Granularity, dtype: int64

## Population

In [330]:
Population.head()

,Country,Granularity,Year,Population
0,Afghanistan,Total,2000,20779.953
1,Afghanistan,Male,2000,10689.508
2,Afghanistan,Female,2000,10090.449
3,Afghanistan,Rural,2000,15657.474
4,Afghanistan,Urban,2000,4436.282


In [331]:
len(Population)

20914

In [332]:
Population['Granularity'].value_counts()

Total     4430
Urban     4414
Rural     4414
Male      3828
Female    3828
Name: Granularity, dtype: int64

Suppression des granularités Male/Female :

In [333]:
Population.drop\
(Population[Population['Granularity'] == 'Male'].index,inplace=True)
Population.drop\
(Population[Population['Granularity'] == 'Female'].index,inplace=True)

Population['Granularity'].value_counts() # vérification

Total    4430
Urban    4414
Rural    4414
Name: Granularity, dtype: int64

## RegionCountry

In [334]:
RegionCountry.head()

,REGION (DISPLAY),COUNTRY (DISPLAY)
0,Europe,Albania
1,Europe,Andorra
2,Europe,Armenia
3,Western Pacific,Australia
4,Europe,Austria


In [335]:
len(RegionCountry)

194

In [336]:
RegionCountry['REGION (DISPLAY)'].value_counts()

Europe                   53
Africa                   47
Americas                 35
Western Pacific          27
Eastern Mediterranean    21
South-East Asia          11
Name: REGION (DISPLAY), dtype: int64

### Changements RegionDisplay

In [337]:
# Pays dans Eastern Meditteranean placés en Afrique
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Morocco', 'Africa' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Tunisia', 'Africa' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Egypt', 'Africa' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Lybia', 'Africa' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Somalia', 'Africa' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Sudan', 'Africa' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Libya', 'Africa' , inplace=True)
# Pays dans Eastern Meditteranean placés en Asie
RegionCountry.loc[RegionCountry['REGION (DISPLAY)'] =='Eastern Mediterranean','REGION (DISPLAY)'] ='Asia'
# Pays dans South-East Asia placés en Asie
RegionCountry.loc[RegionCountry['REGION (DISPLAY)'] =='South-East Asia','REGION (DISPLAY)'] ='Asia'
# Pays dans Western Pacific placés en Océanie
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Australia','Oceania',inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Fiji', 'Oceania',inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Papua New Guinea','Oceania', inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Fiji', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Solomon Islands', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Marshall Islands', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Micronesia (Federated States of)', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'New Zealand', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Vanuatu', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Samoa', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Palau', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Kiribati', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Tuvalu', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Nauru', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Tonga', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Cook Islands', 'Oceania' , inplace=True)
RegionCountry['REGION (DISPLAY)'].mask(RegionCountry['COUNTRY (DISPLAY)'] == 'Niue', 'Oceania' , inplace=True)
# Pays dans Western Pacific placés en Asie
RegionCountry.loc[RegionCountry['REGION (DISPLAY)'] =='Western Pacific','REGION (DISPLAY)'] ='Asia'

In [338]:
RegionCountry[RegionCountry['REGION (DISPLAY)']=='Asia']

,REGION (DISPLAY),COUNTRY (DISPLAY)
6,Asia,Bahrain
7,Asia,Bangladesh
10,Asia,Bhutan
15,Asia,Cambodia
18,Asia,China
26,Asia,Democratic People's Republic of Korea
45,Asia,India
46,Asia,Iran (Islamic Republic of)
47,Asia,Iraq
51,Asia,Japan


In [339]:
RegionCountry['REGION (DISPLAY)'].value_counts()

Africa      53
Europe      53
Asia        37
Americas    35
Oceania     16
Name: REGION (DISPLAY), dtype: int64

# <a name="C4"> Correspondance WaterServices & Mortality </a>

Fusion des datasets sur le pays :

In [340]:
df = BasicAndSafelyManagedDrinkingWaterServices.merge(MortalityRateAttributedToWater,
                                                      how='outer',
                                                      on='Country',
                                                      indicator=True)

Décomptes des lignes sans correspondance avec mention du dataset d'origine :

In [341]:
df._merge.value_counts()

both          9882
left_only      594
right_only       0
Name: _merge, dtype: int64

Tous les pays de Mortality ont une correspondance dans WaterServices.

Affichage des pays dans WaterServices sans correspondance dans Mortality :

In [342]:
left_only = df[df._merge == 'left_only']
left_only['Country'].value_counts()

Nauru                    54
Cook Islands             54
San Marino               54
Monaco                   54
Tuvalu                   54
Marshall Islands         54
Niue                     54
Dominica                 54
Palau                    54
Saint Kitts and Nevis    54
Andorra                  54
Name: Country, dtype: int64

On ne dispose de données sur la mortalité pour ces pays.

Suppression de la colonne _merge :

In [343]:
df.drop(columns=['_merge'],inplace=True)

# <a name="C5"> Correspondance df & PoliticalStability </a>

Fusion des datasets sur le pays :

In [344]:
df = df.merge(PoliticalStability,
             how='outer',
             on='Country',
             indicator=True)

Décomptes des lignes sans correspondance :

In [345]:
df._merge.value_counts()

both          181278
left_only        216
right_only       169
Name: _merge, dtype: int64

Pays dans df sans correspondance dans politicalStability :

In [346]:
left_only = df[df._merge == 'left_only']
left_only['Country'].value_counts()

San Marino                     54
Republic of North Macedonia    54
Monaco                         54
China                          54
Name: Country, dtype: int64

Pays dans PoliticalStability sans correspondance dans df :

In [347]:
right_only = df[df._merge == 'right_only']
right_only['Country'].value_counts()

Palestine                    18
Bermuda                      18
China, Taiwan Province of    18
North Macedonia              18
China, Macao SAR             18
Puerto Rico                  18
China, Hong Kong SAR         18
China, mainland              18
American Samoa               15
Greenland                    10
Name: Country, dtype: int64

## Monaco, San Marino

Pas de données sur la stabilité politique.

## Macédoine 

changement du nom dans PoliticalStability pour établir la correspondance :

In [348]:
PoliticalStability.replace({'North Macedonia': 'Republic of North Macedonia'}, inplace=True)

Vérification :

In [349]:
PoliticalStability[PoliticalStability['Country'] == 'Republic of North Macedonia'].head()

,Country,Year,Political_Stability,Granularity
2320,Republic of North Macedonia,2000,-0.62,Total
2321,Republic of North Macedonia,2002,-1.04,Total
2322,Republic of North Macedonia,2003,-0.99,Total
2323,Republic of North Macedonia,2004,-0.85,Total
2324,Republic of North Macedonia,2005,-1.16,Total


## Puerto Rico, Bermudes, American Samoa, Palestine & Groënland

Aires géographiques avec un indice de stabilité politique qui ne sont pas recensés dans les dataset donnant les indicateurs sur l'eau.

Suppression des indicateurs de stabilité politique car ces aires géographiques prendront l'indicateur de stabilité politique de leurs pays d'appartenance.

In [350]:
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'Puerto Rico'].index,inplace=True)
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'Bermuda'].index,inplace=True)
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'American Samoa'].index,inplace=True)
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'Greenland'].index,inplace=True)
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'Palestine'].index,inplace=True)

Vérifications :

In [351]:
PoliticalStability[PoliticalStability['Country'] == 'Puerto Rico'].head()

,Country,Year,Political_Stability,Granularity


In [352]:
PoliticalStability[PoliticalStability['Country'] == 'Bermuda'].head()

,Country,Year,Political_Stability,Granularity


In [353]:
PoliticalStability[PoliticalStability['Country'] == 'American Samoa'].head()

,Country,Year,Political_Stability,Granularity


In [354]:
PoliticalStability[PoliticalStability['Country'] == 'Greenland'].head()

,Country,Year,Political_Stability,Granularity


In [355]:
PoliticalStability[PoliticalStability['Country'] == 'Palestine'].head()

,Country,Year,Political_Stability,Granularity


## Chine

Suppression de l'indicateur de stabilité politique pour Honk-Kong, Macao et Taïwan car on ne dispose pas de données spécifiques sur l'accès à l'eau pour ces régions chinoises alors que les indicateurs sur l'eau semblent concerner tout le territoire de la Chine ("China").

On est donc obligés, en contrepartie, d'utiliser l'indicateur de stabilité politique de la Chine continentale pour ces zones.

### Suppression pour Honk-Kong :

In [356]:
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'China, Hong Kong SAR'].index,inplace=True)

Vérification :

In [357]:
PoliticalStability[PoliticalStability['Country'] == 'China, Hong Kong SAR']

,Country,Year,Political_Stability,Granularity


### Suppression pour Macao :

In [358]:
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'China, Macao SAR'].index,inplace=True)

Vérification :

In [359]:
PoliticalStability[PoliticalStability['Country'] == 'China, Macao SAR']

,Country,Year,Political_Stability,Granularity


### Suppression pour Taïwan

In [360]:
PoliticalStability.drop(PoliticalStability[PoliticalStability['Country'] == 'China, Taiwan Province of'].index,inplace=True)

Vérification :

In [361]:
PoliticalStability[PoliticalStability['Country'] == 'China, Taiwan Province of']

,Country,Year,Political_Stability,Granularity


### changement du nom 'China, mainland' dans PoliticalStability en 'China' pour établir la correspondance :

In [362]:
PoliticalStability.replace({'China, mainland': 'China'}, inplace=True)

In [363]:
PoliticalStability[PoliticalStability['Country'] == 'China']

,Country,Year,Political_Stability,Granularity
699,China,2000,-0.21,Total
700,China,2002,-0.33,Total
701,China,2003,-0.56,Total
702,China,2004,-0.39,Total
703,China,2005,-0.50,Total
704,China,2006,-0.54,Total
705,China,2007,-0.50,Total
706,China,2008,-0.49,Total
707,China,2009,-0.45,Total
708,China,2010,-0.66,Total


Suppression de la colonne merge :

In [364]:
df.drop(columns=['_merge'],inplace=True)

## Vérification des modifications

Fusion de WaterServices et de Mortality :

In [365]:
df = BasicAndSafelyManagedDrinkingWaterServices.merge(MortalityRateAttributedToWater,
                                                      how='outer',
                                                      on='Country',
                                                      indicator=True)

Suppression de la colonne merge :

In [366]:
df.drop(columns=['_merge'],inplace=True)

Fusion de df et de PoliticalStability :

In [367]:
df = df.merge(PoliticalStability,
              how='outer',
              on='Country',
              indicator=True)

In [368]:
df['_merge'].value_counts()

both          183222
left_only        108
right_only         0
Name: _merge, dtype: int64

Pays dans df sans correspondance dans politicalStability :

In [369]:
left_only = df[df._merge == 'left_only']
left_only['Country'].value_counts()

San Marino    54
Monaco        54
Name: Country, dtype: int64

Vérification validée : il ne reste que les pays où les indicateurs de mortalité sont manquants.

Suppression de la colonne merge :

In [370]:
df.drop(columns=['_merge'],inplace=True)

In [371]:
len(df)

183330

# <a name="C6"> Correspondance df & Population </a>

Fusion des datasets sur le pays :

In [372]:
df = df.merge(Population,
             how='outer',
             on='Country',
             indicator=True)

Décomptes des lignes sans correspondance :

In [373]:
df._merge.value_counts()

both          10313730
right_only        2365
left_only          972
Name: _merge, dtype: int64

Pays dans df sans correspondance dans Population :

In [374]:
left_only = df[df._merge == 'left_only']
left_only['Country'].value_counts()

Republic of North Macedonia    972
Name: Country, dtype: int64

Pays dans Population sans correspondance dans df :

In [375]:
right_only = df[df._merge == 'right_only']
right_only = right_only['Country'].value_counts()
right_only

Greenland                                       57
Montserrat                                      57
China, mainland                                 57
Guam                                            57
Turks and Caicos Islands                        57
United States Virgin Islands                    57
Faroe Islands                                   57
French Guyana                                   57
Anguilla                                        57
Isle of Man                                     57
French Polynesia                                57
Guadeloupe                                      57
Palestine                                       57
China, Macao SAR                                57
Liechtenstein                                   57
Saint Pierre and Miquelon                       57
British Virgin Islands                          57
Netherlands Antilles (former)                   57
Cayman Islands                                  57
American Samoa                 

## Macédoine

Changement du nom 'North Macedonia" en "Republic of North Macedonia" dans Population :

In [376]:
Population.replace({'North Macedonia': 'Republic of North Macedonia'}, inplace=True)

Vérification :

In [377]:
Population[Population['Country'] == 'Republic of North Macedonia'].head()

,Country,Granularity,Year,Population
13904,Republic of North Macedonia,Total,2000,2034.819
13907,Republic of North Macedonia,Rural,2000,843.475
13908,Republic of North Macedonia,Urban,2000,1191.344
13909,Republic of North Macedonia,Total,2001,2042.842
13912,Republic of North Macedonia,Rural,2001,851.059


## Zones géographiques dans population

Ce sont des zones géographiques (pas des pays) dont on a indiqué la population.

La question est de savoir si, dans la population des pays de rattachement, la population de ces zones géographiques ont été ajoutées.

- Si oui : suppression des lignes dans Population
- Si non : Ajout de la popualtion de ces lignes aux pays de rattachement.

1er test sur la population de la Chine. Est-ce que l'entrée "China" prend en compte toute la population chinoise, avec Honk- Kong et Macao ?

In [378]:
Population[(Population['Country'] == 'China') & (Population['Year'] == 2018)]

,Country,Granularity,Year,Population
3876,China,Total,2018,1459377.612
3879,China,Rural,2018,583199.631
3880,China,Urban,2018,863601.691


1 459 377 612 habitants en 2018 selon l'entrée 'China' du datset Population.
- 1 milliard 427 millions selon Worldometer
- 1 milliard 403 millions selon la Banque Mondiale

Conclusion : l'entrée China semble prendre en compte les populations de Honk-Kong et de Macao.

2ème test sur la population du Royaume-Uni.

In [379]:
Population[(Population['Country'] == 'United Kingdom of Great Britain and Northern Ireland') \
           & (Population['Year'] == 2018)]

,Country,Granularity,Year,Population
19712,United Kingdom of Great Britain and Northern I...,Total,2018,67141.684
19715,United Kingdom of Great Britain and Northern I...,Rural,2018,11052.278
19716,United Kingdom of Great Britain and Northern I...,Urban,2018,55521.226


67 millions selon l'entrée Royaume-Uni du dataset Population.
- 66,46 millions selon la Banque mondiale.

Conclusion : l'entrée Royaume-Uni semble prendre en compte les aires géographiques mentionnées par ailleurs dans le dataset Population (Iles Vierges Britanniques par exemple).

Conclusion Générale : suppression des lignes du dataset Population sans correspondance dans dataset df.

## Suppression des lignes dans Population

In [380]:
type(right_only)

pandas.core.series.Series

Transformation de l'index de right_only en liste (contenant les noms des aires géographiques à supprimer) :

In [381]:
right_only = right_only.index.tolist()

Vérification :

In [382]:
type(right_only)

list

In [383]:
print(right_only)

['Greenland', 'Montserrat', 'China, mainland', 'Guam', 'Turks and Caicos Islands', 'United States Virgin Islands', 'Faroe Islands', 'French Guyana', 'Anguilla', 'Isle of Man', 'French Polynesia', 'Guadeloupe', 'Palestine', 'China, Macao SAR', 'Liechtenstein', 'Saint Pierre and Miquelon', 'British Virgin Islands', 'Netherlands Antilles (former)', 'Cayman Islands', 'American Samoa', 'Mayotte', 'Saint Helena, Ascension and Tristan da Cunha', 'Gibraltar', 'Falkland Islands (Malvinas)', 'Martinique', 'New Caledonia', 'Wallis and Futuna Islands', 'Holy See', 'Bermuda', 'Channel Islands', 'Aruba', 'Western Sahara', 'Puerto Rico', 'North Macedonia', 'Réunion', 'Northern Mariana Islands', 'China, Hong Kong SAR', 'China, Taiwan Province of', 'Tokelau', 'Sudan (former)', 'Curaçao', 'Sint Maarten  (Dutch part)', 'Bonaire, Sint Eustatius and Saba', 'Serbia and Montenegro', 'Saint Barthélemy', 'Saint-Martin (French part)']


Suppression des entrées dans Population :

In [384]:
for i in right_only :
    Population.drop(Population[Population['Country'] == i].index,inplace=True)

Vérification :

In [385]:
for i in right_only :
    print(Population[Population['Country'] == i])

Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: []
Empty DataFr

Suppression de la colonne merge :

In [386]:
df.drop(columns=['_merge'],inplace=True)

## Vérification des modifications :

Fusion de WaterServices et de Mortality :

In [387]:
df = BasicAndSafelyManagedDrinkingWaterServices.merge(MortalityRateAttributedToWater,
                                                      how='outer',
                                                      on='Country',
                                                      indicator=True)

Suppression de la colonne merge :

In [388]:
df.drop(columns=['_merge'],inplace=True)

Fusion de df et de PoliticalStability :

In [389]:
df = df.merge(PoliticalStability,
              how='outer',
              on='Country',
              indicator=True)

In [390]:
df['_merge'].value_counts()

both          183222
left_only        108
right_only         0
Name: _merge, dtype: int64

Suppression de la colonne merge :

In [391]:
df.drop(columns=['_merge'],inplace=True)

Fusion de df et de Population :

In [392]:
df = df.merge(Population,
              how='outer',
              on='Country',
              indicator=True)

In [393]:
df['_merge'].value_counts()

both          10369134
right_only           0
left_only            0
Name: _merge, dtype: int64

Vérification validée.

Suppression de la colonne merge :

In [394]:
df.drop(columns=['_merge'],inplace=True)

# <a name="C7"> Correspondance df & RegionCountry </a>

In [395]:
RegionCountry.head()

,REGION (DISPLAY),COUNTRY (DISPLAY)
0,Europe,Albania
1,Europe,Andorra
2,Europe,Armenia
3,Oceania,Australia
4,Europe,Austria


Il faut renommer la colonne 'COUNTRY (DISPLAY)' en 'Country' pour la fusion des données.

In [396]:
RegionCountry.rename(columns={"COUNTRY (DISPLAY)":"Country"}, inplace=True)

Vérification :

In [397]:
RegionCountry.head()

,REGION (DISPLAY),Country
0,Europe,Albania
1,Europe,Andorra
2,Europe,Armenia
3,Oceania,Australia
4,Europe,Austria


Fusion de df et de RegionCountry :

In [398]:
df = df.merge(RegionCountry,
              how='outer',
              on='Country',
              indicator=True)

In [399]:
df['_merge'].value_counts()

both          10369134
right_only           0
left_only            0
Name: _merge, dtype: int64

Toutes les entrées 'Country' correspondent.

# <a name="C8"> Fusion des datsets après nettoyage </a>

## <a name="C9"> Fusion WaterServices & Mortality </a>

Analyses des colonnes sur lesquelles fusionner :

In [400]:
BasicAndSafelyManagedDrinkingWaterServices.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)
0,2000,Afghanistan,Rural,21.61913,NaN
1,2000,Afghanistan,Total,27.77190,NaN
2,2000,Afghanistan,Urban,49.48745,NaN
3,2000,Albania,Rural,81.78472,NaN
4,2000,Albania,Total,87.86662,49.29324


In [401]:
MortalityRateAttributedToWater.head()

,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
2,2016,Afghanistan,Total,13.92067,4824.35300
5,2016,Albania,Total,0.16641,4.86975
8,2016,Algeria,Total,1.86723,758.21000
11,2016,Angola,Total,48.81467,14065.20000
14,2016,Antigua and Barbuda,Total,0.11403,0.11513


In [402]:
temp_df = pd.pivot_table(MortalityRateAttributedToWater,
                         values=['WASH deaths'],
                         aggfunc='sum',
                         index='Granularity')
temp_df

,WASH deaths
Granularity,
Total,870365.88011


Fusion sur les colonnes Country, year et Granularity :

In [403]:
df = BasicAndSafelyManagedDrinkingWaterServices.merge(MortalityRateAttributedToWater,
                                                      how='outer',
                                                      on=['Country','Year','Granularity'],
                                                      indicator=True)

Vérification de la fusion

In [404]:
df['_merge'].value_counts()

left_only     10293
both            183
right_only        0
Name: _merge, dtype: int64

183 lignes ont une correspondance. Nombre de lignes du dataset MortalityRateAttributedToWater :

In [405]:
len(MortalityRateAttributedToWater)

183

Toutes les données ont été fusionnées. Vérification de la fusion en filtrant sur l'année 2016, la granularité Total et l'indicateur _merge à 'both'.

On doit trouver 183 lignes pour être sur que ça ait été fusionné sur l'année 2016 et la granularité Total :

In [406]:
temp_df = df[(df['Year'] == 2016) & (df['Granularity'] == 'Total') & (df['_merge'] == 'both')]
print('Nombre de lignes :',len(temp_df))

Nombre de lignes : 183


In [407]:
temp_df

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,_merge
9313,2016,Afghanistan,Total,64.28648,NaN,13.92067,4824.35300,both
9316,2016,Albania,Total,91.02037,69.98433,0.16641,4.86975,both
9319,2016,Algeria,Total,93.51585,NaN,1.86723,758.21000,both
9325,2016,Angola,Total,55.08428,NaN,48.81467,14065.20000,both
9328,2016,Antigua and Barbuda,Total,96.73919,NaN,0.11403,0.11513,both
9331,2016,Argentina,Total,99.07838,NaN,0.36294,159.14110,both
9334,2016,Armenia,Total,99.89854,83.00597,0.17688,5.17336,both
9337,2016,Australia,Total,99.96997,NaN,0.09539,23.01346,both
9340,2016,Austria,Total,100.00000,98.90781,0.12375,10.78108,both
9343,2016,Azerbaijan,Total,90.71031,73.01500,1.11988,108.91280,both


Vérification validée.

Suppression de la colonne _merge

In [408]:
df.drop('_merge', inplace=True, axis=1)

Vérification de la suppression

In [409]:
df.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN
3,2000,Albania,Rural,81.78472,NaN,NaN,NaN
4,2000,Albania,Total,87.86662,49.29324,NaN,NaN


## <a name="C10"> Fusion df & Political Stability </a>

Analyse des colonnes de PoliticalStability pour savoir sur quelles colonnes fusionner :

In [410]:
PoliticalStability.head()

,Country,Year,Political_Stability,Granularity
0,Afghanistan,2000,-2.44,Total
1,Afghanistan,2002,-2.04,Total
2,Afghanistan,2003,-2.20,Total
3,Afghanistan,2004,-2.30,Total
4,Afghanistan,2005,-2.07,Total


Il faut fusionner sur Country, Year et Granularity.

Recherche des granularités présentes dans Political Stability :

In [411]:
PoliticalStability['Granularity'].value_counts()

Total    3393
Name: Granularity, dtype: int64

Il n'y a que la granularité Total.

Fusion des datasets :

In [412]:
df1 = df.merge(PoliticalStability,
              how='outer',
              on=['Country','Year','Granularity'],
              indicator=True)

Vérification de la fusion :

In [413]:
df1['_merge'].value_counts()

left_only     7273
both          3203
right_only     190
Name: _merge, dtype: int64

In [414]:
right_only = df1[df1['_merge'] == 'right_only']
right_only.head(20)

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,_merge
10476,2018,Afghanistan,Total,NaN,NaN,NaN,NaN,-2.75,right_only
10477,2018,Albania,Total,NaN,NaN,NaN,NaN,0.38,right_only
10478,2018,Algeria,Total,NaN,NaN,NaN,NaN,-0.79,right_only
10479,2018,Andorra,Total,NaN,NaN,NaN,NaN,1.43,right_only
10480,2018,Angola,Total,NaN,NaN,NaN,NaN,-0.32,right_only
10481,2018,Antigua and Barbuda,Total,NaN,NaN,NaN,NaN,0.73,right_only
10482,2018,Argentina,Total,NaN,NaN,NaN,NaN,0.02,right_only
10483,2018,Armenia,Total,NaN,NaN,NaN,NaN,-0.42,right_only
10484,2018,Australia,Total,NaN,NaN,NaN,NaN,0.98,right_only
10485,2018,Austria,Total,NaN,NaN,NaN,NaN,0.92,right_only


In [415]:
right_only['Year'].value_counts()

2018    190
Name: Year, dtype: int64

Pas de fusion sur l'année 2018. Y'a-t-il une année 2018 dans df ?

In [416]:
df['Year'].value_counts().sort_index()

2000    582
2001    582
2002    582
2003    582
2004    582
2005    582
2006    582
2007    582
2008    582
2009    582
2010    582
2011    582
2012    582
2013    582
2014    582
2015    582
2016    582
2017    582
Name: Year, dtype: int64

Non, les right_only sont donc justifiés.

Il y a 3393 lignes dans le dataset PoliticalStability. 3203 lignes ont été fusionnées et 190 non (année 2018 manquante dans df). Toutes les lignes ont-elles été vérifiées ?

In [417]:
3203+190

3393

Oui.

Vérification de la fusion df-PoliticalStability validée.

In [418]:
df1.drop(columns=['_merge'],inplace=True)

## <a name="C11"> Fusion df & Population </a>

Lecture des colonnes de Population pour savoir sur quelles colonnes fusionner :

In [419]:
Population.head()

,Country,Granularity,Year,Population
0,Afghanistan,Total,2000,20779.953
3,Afghanistan,Rural,2000,15657.474
4,Afghanistan,Urban,2000,4436.282
5,Afghanistan,Total,2001,21606.988
8,Afghanistan,Rural,2001,16318.324


Il faut fusionner sur Country, Year et Granularity.

Recherche des granularités présentes dans Population :

In [420]:
Population['Granularity'].value_counts()

Urban    3650
Rural    3650
Total    3650
Name: Granularity, dtype: int64

Fusion des datasets :

In [421]:
df2 = df1.merge(Population,
              how='outer',
              on=['Country','Year','Granularity'],
              indicator=True)

Vérification de la fusion :

In [422]:
df2['_merge'].value_counts()

both          10558
right_only      392
left_only       108
Name: _merge, dtype: int64

In [423]:
right_only = df2[df2['_merge'] == 'right_only']
right_only.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,_merge
10666,2018,Afghanistan,Rural,NaN,NaN,NaN,NaN,NaN,27099.874,right_only
10667,2018,Afghanistan,Urban,NaN,NaN,NaN,NaN,NaN,9273.302,right_only
10668,2018,Albania,Rural,NaN,NaN,NaN,NaN,NaN,1164.388,right_only
10669,2018,Albania,Urban,NaN,NaN,NaN,NaN,NaN,1769.975,right_only
10670,2018,Algeria,Rural,NaN,NaN,NaN,NaN,NaN,11498.038,right_only


Données de 2018 ?

In [424]:
right_only['Year'].value_counts()

2018    392
Name: Year, dtype: int64

Le dataset df1 ne contient, pour l'année 2018, que la granularité Total puisque :
- df (WaterServices + Mortality) n'a pas de données sur 2018
- PoliticalStability a des données sur 2018 sur la granularité Total

On s'attend donc à ne trouver que les granularités Rural et Urban sur right_only.

In [425]:
right_only['Granularity'].value_counts()

Rural    194
Urban    194
Total      4
Name: Granularity, dtype: int64

Affichage des granularités Total :

In [426]:
right_only[right_only['Granularity'] == 'Total']

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,_merge
10744,2018,Cook Islands,Total,NaN,NaN,NaN,NaN,NaN,17.518,right_only
10891,2018,Monaco,Total,NaN,NaN,NaN,NaN,NaN,38.682,right_only
10920,2018,Niue,Total,NaN,NaN,NaN,NaN,NaN,1.620,right_only
10967,2018,San Marino,Total,NaN,NaN,NaN,NaN,NaN,33.785,right_only


Hypothèse : les pays dans right_only n'existent pas dans PoliticalStability, au moins pour l'année 2018.

Vérification :

In [427]:
countries = right_only[right_only['Granularity'] == 'Total']
countries_list = countries['Country']
countries_list

10744    Cook Islands
10891          Monaco
10920            Niue
10967      San Marino
Name: Country, dtype: object

In [428]:
type(countries_list)

pandas.core.series.Series

In [429]:
for i in countries_list :
    print(PoliticalStability[PoliticalStability['Country'] == i])

          Country  Year  Political_Stability Granularity
789  Cook Islands  2009                 1.14       Total
790  Cook Islands  2010                 1.32       Total
Empty DataFrame
Columns: [Country, Year, Political_Stability, Granularity]
Index: []
     Country  Year  Political_Stability Granularity
2318    Niue  2009                 1.14       Total
2319    Niue  2010                 1.32       Total
Empty DataFrame
Columns: [Country, Year, Political_Stability, Granularity]
Index: []


Ces pays n'ont pas de données pour l'année 2018, ce qui explique qu'il n'y ait pas de fusion.

Vérification right_only validée.

Nombre de lignes de Population :

In [430]:
len(Population)

10950

Résultat de la fusion df1 + Population

In [431]:
df2['_merge'].value_counts()

both          10558
right_only      392
left_only       108
Name: _merge, dtype: int64

Si les lignes avec indicateurs both + right_only = 10950, toutes les lignes auront été vérifiées.

In [432]:
392+10558

10950

Fusion df1 + Population validée.

Suppression colonne _merge :

In [433]:
df2.drop(columns = '_merge', inplace=True)

## <a name="C12"> Fusion df2 & RegionCountry </a>

Affichage des colonnes de RegionCountry pour savoir sur quelles colonnes fusionner :

In [434]:
RegionCountry.head()

,REGION (DISPLAY),Country
0,Europe,Albania
1,Europe,Andorra
2,Europe,Armenia
3,Oceania,Australia
4,Europe,Austria


Fusion sur Country :

In [435]:
df3 = df2.merge(RegionCountry,
              how='outer',
              on='Country',
              indicator=True)

Vérification de la fusion :

In [436]:
df3['_merge'].value_counts()

both          11058
right_only        0
left_only         0
Name: _merge, dtype: int64

Fusion validée

In [437]:
df3.drop(columns = '_merge', inplace=True)

In [438]:
df3.head(50)

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY)
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia
5,2001,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4648.139,Asia
6,2002,Afghanistan,Rural,23.59988,NaN,NaN,NaN,NaN,17086.910,Asia
7,2002,Afghanistan,Total,29.90076,NaN,NaN,NaN,-2.04,22600.770,Asia
8,2002,Afghanistan,Urban,51.90447,NaN,NaN,NaN,NaN,4893.013,Asia
9,2003,Afghanistan,Rural,25.58063,NaN,NaN,NaN,NaN,17909.063,Asia


In [439]:
df3.to_csv('df.csv', index=False)

In [440]:
df4 = df3

# <a name="C14.2"> Ajout de la colonne Urban population rate </a>

Stockage granularité Total :

In [441]:
total = df4[df4['Granularity'] == 'Total']
total.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY)
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia
7,2002,Afghanistan,Total,29.90076,NaN,NaN,NaN,-2.04,22600.770,Asia
10,2003,Afghanistan,Total,32.00507,NaN,NaN,NaN,-2.20,23680.871,Asia
13,2004,Afghanistan,Total,34.12623,NaN,NaN,NaN,-2.30,24726.684,Asia


Stockage granularité Urban :

In [442]:
urban = df4[df4['Granularity'] == 'Urban']
urban.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY)
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia
5,2001,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4648.139,Asia
8,2002,Afghanistan,Urban,51.90447,NaN,NaN,NaN,NaN,4893.013,Asia
11,2003,Afghanistan,Urban,54.32149,NaN,NaN,NaN,NaN,5155.788,Asia
14,2004,Afghanistan,Urban,56.73851,NaN,NaN,NaN,NaN,5426.872,Asia


In [443]:
df5 = df4.merge(urban[['Country','Year','Population']],
                  how='left',
                  on=['Country','Year'],
                  indicator=True)
df5['_merge'].value_counts()

both          11058
right_only        0
left_only         0
Name: _merge, dtype: int64

In [444]:
df5.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population_x,REGION (DISPLAY),Population_y,_merge
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282,both
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282,both
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282,both
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139,both
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139,both


In [445]:
df5.drop(columns='_merge',inplace=True)

In [446]:
df5.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population_x,REGION (DISPLAY),Population_y
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139


In [447]:
df5.rename(columns={"Population_x": "Population","Population_y": "Urban population"},inplace=True)

In [448]:
df5.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139


In [449]:
df5.loc[df5['Granularity'] == 'Total', 'Urban population rate'] = \
(df5['Urban population']/df5['Population'])*100

In [450]:
df5.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population,Urban population rate
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282,NaN
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282,21.348855
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282,NaN
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139,NaN
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139,21.512202


# <a name="C14.5"> Ajout des densités de Population </a>

## <a name="C14.51"> Ajout des densités de Population par Pays</a>

In [451]:
pop_density_country = pd.read_excel('population density Country calculated from World Bank.xlsx', index_col=0)
pop_density_country.head()

,Country,Granularity,Population density
Year,,,
2000,Afghanistan,Total,31.829117
2001,Afghanistan,Total,33.095904
2002,Afghanistan,Total,34.618102
2003,Afghanistan,Total,36.272510
2004,Afghanistan,Total,37.874413


In [452]:
df6 = df5.merge(pop_density_country,
                how='left',
                on=['Country','Year','Granularity'],
                indicator=True)

In [453]:
df6['_merge'].value_counts()

left_only     7421
both          3637
right_only       0
Name: _merge, dtype: int64

Les granularités Urban et Rural n'ont pas pu être jointées.

In [454]:
Urban = df6[df6['Granularity'] == 'Urban']
Rural = df6[df6['Granularity'] == 'Rural']
len(Urban)+len(Rural)

7372

Sur les 7 421 'left_only', 7 372 lignes justifiées par une granularité Rural ou Urban.

Combien de 'left_only' reste-t-il à justifier ?

In [455]:
7421-7372

49

In [456]:
len(df6[(df6['Granularity'] == 'Total') & (df6['_merge'] == 'left_only')])

49

La sélection comprend bien les 49 lignes restant à justifier.

In [457]:
df6[(df6['Granularity'] == 'Total') & (df6['_merge'] == 'left_only')]

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population,Urban population rate,Population density,_merge
2224,2000,Cook Islands,Total,99.89700,NaN,NaN,NaN,NaN,17.930,Oceania,11.814,65.889571,NaN,left_only
2227,2001,Cook Islands,Total,99.89565,NaN,NaN,NaN,NaN,18.025,Oceania,12.258,68.005548,NaN,left_only
2230,2002,Cook Islands,Total,99.89432,NaN,NaN,NaN,NaN,18.278,Oceania,12.750,69.755991,NaN,left_only
2233,2003,Cook Islands,Total,99.89298,NaN,NaN,NaN,NaN,18.608,Oceania,13.239,71.146819,NaN,left_only
2236,2004,Cook Islands,Total,99.89164,NaN,NaN,NaN,NaN,18.904,Oceania,13.684,72.386796,NaN,left_only
2239,2005,Cook Islands,Total,99.89030,NaN,NaN,NaN,NaN,19.074,Oceania,13.999,73.393101,NaN,left_only
2242,2006,Cook Islands,Total,99.88897,NaN,NaN,NaN,NaN,19.092,Oceania,14.170,74.219568,NaN,left_only
2245,2007,Cook Islands,Total,99.88763,NaN,NaN,NaN,NaN,19.001,Oceania,14.140,74.417136,NaN,left_only
2248,2008,Cook Islands,Total,99.88629,NaN,NaN,NaN,NaN,18.818,Oceania,13.974,74.258688,NaN,left_only
2251,2009,Cook Islands,Total,99.88495,NaN,NaN,NaN,1.14,18.602,Oceania,13.773,74.040426,NaN,left_only


Pas de données sur Cook Islands et Inue. Données parcellaires sur Soudan du Sud.

Fusion validée.

In [458]:
df6.drop(columns='_merge', inplace=True)

In [459]:
df6 = df6.rename(columns={'Population density':'Population density Country'})

In [460]:
df6['REGION (DISPLAY)'].value_counts()

Europe      3021
Africa      3021
Asia        2109
Americas    1995
Oceania      912
Name: REGION (DISPLAY), dtype: int64

## <a name="C14.52"> Ajout des densités de Population par Région</a>

Données provenant de https://www.populationdata.net/monde/

In [461]:
# create a list of our conditions
conditions = [
    (df6['REGION (DISPLAY)'] == 'Asia'),
    (df6['REGION (DISPLAY)'] == 'Africa'),
    (df6['REGION (DISPLAY)'] == 'Americas'),
    (df6['REGION (DISPLAY)'] == 'Oceania'),
    (df6['REGION (DISPLAY)'] == 'Europe'),
    ]

# create a list of the values we want to assign for each condition
values = [142.7, 43.25, 24.29, 5.06, 32.32]

# create a new column and use np.select to assign values to it using our lists as arguments
df6['population density region'] = np.select(conditions, values)

# display updated DataFrame
df6['population density region'].value_counts()

43.25     3021
32.32     3021
142.70    2109
24.29     1995
5.06       912
Name: population density region, dtype: int64

In [462]:
df6.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population,Urban population rate,Population density Country,population density region
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282,NaN,NaN,142.7
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282,21.348855,31.829117,142.7
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282,NaN,NaN,142.7
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139,NaN,NaN,142.7
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139,21.512202,33.095904,142.7


In [463]:
df7 = df6

## <a name="C14.53"> Ajout de la densité de Population au niveau Monde</a>

In [464]:
df7['population density world'] = 50.2

In [465]:
df7.head()

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population,Urban population rate,Population density Country,population density region,population density world
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282,NaN,NaN,142.7,50.2
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282,21.348855,31.829117,142.7,50.2
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282,NaN,NaN,142.7,50.2
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139,NaN,NaN,142.7,50.2
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139,21.512202,33.095904,142.7,50.2


# <a name="C15"> Export du .csv </a>

In [466]:
df7.to_csv('df.csv', index=False)

In [467]:
df7[df7['Country'] == 'China']

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population,Urban population rate,Population density Country,population density region,population density world
1995,2000,China,Rural,70.40273,NaN,NaN,NaN,NaN,829403.316,Asia,482727.522,NaN,NaN,142.7,50.2
1996,2000,China,Total,80.39258,NaN,NaN,NaN,-0.21,1319551.401,Asia,482727.522,36.582699,133.971904,142.7,50.2
1997,2000,China,Urban,98.24744,93.28438,NaN,NaN,NaN,482727.522,Asia,482727.522,NaN,NaN,142.7,50.2
1998,2001,China,Rural,70.40273,NaN,NaN,NaN,NaN,818630.475,Asia,501452.843,NaN,NaN,142.7,50.2
1999,2001,China,Total,80.73117,NaN,NaN,NaN,NaN,1328341.175,Asia,501452.843,37.750305,134.948590,142.7,50.2
2000,2001,China,Urban,98.24744,93.28438,NaN,NaN,NaN,501452.843,Asia,501452.843,NaN,NaN,142.7,50.2
2001,2002,China,Rural,71.40005,NaN,NaN,NaN,NaN,806104.331,Asia,521893.970,NaN,NaN,142.7,50.2
2002,2002,China,Total,81.70317,NaN,NaN,NaN,-0.33,1336765.152,Asia,521893.970,39.041560,135.855780,142.7,50.2
2003,2002,China,Urban,98.21362,93.22449,NaN,NaN,NaN,521893.970,Asia,521893.970,NaN,NaN,142.7,50.2
2004,2003,China,Rural,72.39563,NaN,NaN,NaN,NaN,793136.597,Asia,542756.370,NaN,NaN,142.7,50.2


In [468]:
df7.head(50)

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%),Mortality rate attributed to exposure to unsafe WASH services,WASH deaths,Political_Stability,Population,REGION (DISPLAY),Urban population,Urban population rate,Population density Country,population density region,population density world
0,2000,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,15657.474,Asia,4436.282,NaN,NaN,142.7,50.2
1,2000,Afghanistan,Total,27.77190,NaN,NaN,NaN,-2.44,20779.953,Asia,4436.282,21.348855,31.829117,142.7,50.2
2,2000,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4436.282,Asia,4436.282,NaN,NaN,142.7,50.2
3,2001,Afghanistan,Rural,21.61913,NaN,NaN,NaN,NaN,16318.324,Asia,4648.139,NaN,NaN,142.7,50.2
4,2001,Afghanistan,Total,27.79726,NaN,NaN,NaN,NaN,21606.988,Asia,4648.139,21.512202,33.095904,142.7,50.2
5,2001,Afghanistan,Urban,49.48745,NaN,NaN,NaN,NaN,4648.139,Asia,4648.139,NaN,NaN,142.7,50.2
6,2002,Afghanistan,Rural,23.59988,NaN,NaN,NaN,NaN,17086.910,Asia,4893.013,NaN,NaN,142.7,50.2
7,2002,Afghanistan,Total,29.90076,NaN,NaN,NaN,-2.04,22600.770,Asia,4893.013,21.649762,34.618102,142.7,50.2
8,2002,Afghanistan,Urban,51.90447,NaN,NaN,NaN,NaN,4893.013,Asia,4893.013,NaN,NaN,142.7,50.2
9,2003,Afghanistan,Rural,25.58063,NaN,NaN,NaN,NaN,17909.063,Asia,5155.788,NaN,NaN,142.7,50.2
